In [2]:
import numpy as np

In [8]:
def joint_multifractal(p,q, n):
    P = p.copy()
    Q = q.copy()
    for i in range(n):
        P = np.kron(P, p)
        Q = np.kron(Q, q)
    return(np.stack([P,Q], axis=-1))

rng = np.random.default_rng()

def joint_logmultifractal(n):
    P = rng.lognormal(0, 1, (2,2))
    Q = rng.lognormal(0, 1, (2,2))
    for i in range(n):
        P = np.kron(P, np.ones((2,2))) 
        P = P * rng.lognormal(0, 1, P.shape)
        Q = np.kron(Q, np.ones((2,2))) 
        Q = Q * rng.lognormal(0, 1, Q.shape)
    return(np.stack([P,Q], axis=-1))
    

In [10]:
cascade_simple = joint_multifractal(np.array([[0.25,0.25],[0.1,0.4]]),
                   np.array([[0.1,0.1],[0.7,0.1]]),
                   8)

In [11]:
cascade_lognormal = joint_logmultifractal(8)

In [13]:
np.savez('cascade_simple_log.npz',  cascade_simple, cascade_lognormal)

In [45]:
l=-.1
W = np.exp(rng.multivariate_normal(np.array([0,0]), np.array([[1,l], [l, 1]]), 100000))

In [46]:
W

array([[1.22878814, 0.50215377],
       [1.10482285, 1.38683679],
       [0.25990439, 0.89235613],
       ...,
       [1.3929147 , 0.38764565],
       [0.63022876, 0.2805087 ],
       [3.2555646 , 2.1795079 ]])

In [47]:
np.corrcoef(W.T)

array([[ 1.       , -0.0588691],
       [-0.0588691,  1.       ]])

In [27]:
import numpy as np

A = [45, 37, 42, 35, 39]
B = [38, 31, 26, 28, 33]
C = [10, 15, 17, 21, 12]

data = np.array([A, B, C])

cov_matrix = np.corrcoef(data)
print(cov_matrix)

[[ 1.          0.5184571  -0.70188642]
 [ 0.5184571   1.         -0.86094096]
 [-0.70188642 -0.86094096  1.        ]]


In [30]:
data

array([[45, 37, 42, 35, 39],
       [38, 31, 26, 28, 33],
       [10, 15, 17, 21, 12]])